This is where we are creating the data frames for stack spectrograms

In [123]:
import os
import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
from skimage.transform import resize
import io


In [124]:
#Listar os vetores que possuem a terminação
folder = 'test'


In [125]:

# Definindo os parâmetros do STFT
FRAME_SIZE = 2048
HOP_SIZE = 512

# Intervalo de frequências
fmin = 15000  # 15kHz
fmax = 48000  # 48kHz

# Diretórios
base_dir = os.path.join(f'E:/backup_for_copy/{folder}/')
subdirs = ['positive', 'negative']
#subdirs = ['negative']
save_dir = 'C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/'
chunks_data = pd.read_csv('C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/chunk_labels.csv')

# Configurações da imagem
dpi = 300
width, height = 224 / dpi, 224 / dpi

In [126]:

#Criando coluna 
# Criar a nova coluna concatenando o nome final do arquivo com audio_filename
chunks_data['final_file_name'] = chunks_data.apply(
    lambda row: f"{os.path.basename(row['chunk_file_name']).replace('.wav', '')}_{row['audio_filename'].replace('.wav', '')}", 
    axis=1
)

# Adicionar a extensão .wav ao final
chunks_data['final_file_name'] = chunks_data['final_file_name'] 

chunks_data

,audio_filename,original_file_length_seconds,original_file_sample_rate,chunk_index,chunk_start_seconds,chunk_end_seconds,chunk_file_name,label,call_begin_time,call_end_time,call_length_seconds,call_channel,call_length_within_chunk,final_file_name
0,LPS1142017_MF_20170808_051031_489.wav,568.511000,96000,334,83.50,85.50,E:\all_audios\chunks\LPS1142017_MF_20170808_05...,whistle,85.038926,85.327807,0.288881,4.0,0.288881,chunk_334_LPS1142017_MF_20170808_051031_489
1,LPS1142017_MF_20170808_051031_489.wav,568.511000,96000,666,166.50,168.50,E:\all_audios\chunks\LPS1142017_MF_20170808_05...,click,166.786628,166.989626,0.202998,4.0,0.202998,chunk_666_LPS1142017_MF_20170808_051031_489
2,LPS1142017_MF_20170808_051031_489.wav,568.511000,96000,1076,269.00,271.00,E:\all_audios\chunks\LPS1142017_MF_20170808_05...,click,269.473570,269.752371,0.278801,4.0,0.278801,chunk_1076_LPS1142017_MF_20170808_051031_489
3,LPS1142017_MF_20170808_051031_489.wav,568.511000,96000,1091,272.75,274.75,E:\all_audios\chunks\LPS1142017_MF_20170808_05...,whistle,273.677669,274.042042,0.364373,4.0,0.364373,chunk_1091_LPS1142017_MF_20170808_051031_489
4,LPS1142017_MF_20170808_051031_489.wav,568.511000,96000,1190,297.50,299.50,E:\all_audios\chunks\LPS1142017_MF_20170808_05...,whistle,298.071408,298.667656,0.596248,4.0,0.596248,chunk_1190_LPS1142017_MF_20170808_051031_489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74321,PAM_MF_20190204_064000_000.wav,582.542219,96000,2287,571.75,573.75,E:\all_audios\chunks\PAM_MF_20190204_064000_00...,no_call,NaN,NaN,NaN,NaN,NaN,chunk_2287_PAM_MF_20190204_064000_000
74322,PAM_MF_20190204_064000_000.wav,582.542219,96000,2294,573.50,575.50,E:\all_audios\chunks\PAM_MF_20190204_064000_00...,no_call,NaN,NaN,NaN,NaN,NaN,chunk_2294_PAM_MF_20190204_064000_000
74323,PAM_MF_20190204_064000_000.wav,582.542219,96000,2302,575.50,577.50,E:\all_audios\chunks\PAM_MF_20190204_064000_00...,no_call,NaN,NaN,NaN,NaN,NaN,chunk_2302_PAM_MF_20190204_064000_000
74324,PAM_MF_20190204_064000_000.wav,582.542219,96000,2303,575.75,577.75,E:\all_audios\chunks\PAM_MF_20190204_064000_00...,no_call,NaN,NaN,NaN,NaN,NaN,chunk_2303_PAM_MF_20190204_064000_000


#TESTE

In [127]:


# Função para gerar espectrogramas e salvar em um array
def generate_spectrogram(audio_path):
    # Extrair o nome do arquivo sem a extensão .wav
    file_name = os.path.basename(audio_path).replace('.wav', '')
    # Encontrar a linha correspondente no DataFrame
    row = chunks_data[chunks_data['final_file_name'].str.contains(file_name)]

    # Verificar se o valor de 'call_channel' não é NaN
    if pd.notna(row['call_channel'].values[0]):
        call_channel = int(row['call_channel'].values[0] - 1)  # Subtrai 1 para ajustar o índice
    else:
        call_channel = int(0)  # Caso seja NaN, atribui o canal de índice 0


    y, sr = librosa.load(audio_path, sr=None, mono=False)
    y = y[call_channel,:]
    stft = librosa.stft(y, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
    y_scale = np.abs(stft) ** 2
    y_log_scale = librosa.power_to_db(y_scale)

    plt.figure(figsize=(width, height), dpi=dpi)
    plt.axis('off')
    plt.margins(0, 0)
    plt.ylim(fmin, fmax)

    librosa.display.specshow(y_log_scale[ :, :],
                                sr=sr,
                                hop_length=HOP_SIZE,
                                x_axis='time',
                                y_axis='linear',
                                cmap='gray',
                                fmin=fmin,
                                fmax=fmax)

    plt.clim(-60, 10)
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.tight_layout(pad=0)
    # Salvar a imagem temporariamente em um array
    #plt.savefig(f'single.png', bbox_inches='tight', pad_inches=0, dpi=dpi)
    # Salvar a figura em um buffer de memória
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0, dpi=dpi)
    buf.seek(0)

    # Ler a imagem do buffer e redimensionar
    img = plt.imread(buf)
    img = img[..., :3] #Isso é para transformar os canais RGBA em apenas RGB exatamente como a CNN aceita
    img_resized = resize(img, (224, 224), anti_aliasing=True)


    print(f'O shape da imagem é {img_resized.shape}')


    buf.close()
    plt.close()

    return img_resized

# Processar os áudios em lotes e salvar os espectrogramas
batch_size = 10  # Tamanho do lote

for subdir in subdirs:
    path = os.path.join(base_dir, subdir)
    files = [f for f in os.listdir(path) if f.endswith('.wav')]
    
    for i in range(0, len(files), batch_size):
        batch_files = files[i:i+batch_size]
        spectrogram_list = []
        
        for file in batch_files:
            audio_path = os.path.join(path, file)
            espectrograma = generate_spectrogram(audio_path)
            print(f'O espectrograma salvo é no shape {espectrograma.shape}')
            spectrogram_list.append(espectrograma)
        
        # Criar o nome do arquivo baseado no número do lote
        batch_index = i // batch_size + 1
        spectrogram_array = np.stack(spectrogram_list, axis=-1)
        print(f'o arquivo em lote np salvo é no shape ...{spectrogram_array.shape}')
        np.save(os.path.join(save_dir, f'gray_spectr_{folder}_{subdir}_batch_{batch_index}.npy'), spectrogram_array)

print('Processamento concluído!')


O shape da imagem é (224, 224, 3)
O espectrograma salvo é no shape (224, 224, 3)
O shape da imagem é (224, 224, 3)
O espectrograma salvo é no shape (224, 224, 3)
O shape da imagem é (224, 224, 3)
O espectrograma salvo é no shape (224, 224, 3)
O shape da imagem é (224, 224, 3)
O espectrograma salvo é no shape (224, 224, 3)
O shape da imagem é (224, 224, 3)
O espectrograma salvo é no shape (224, 224, 3)
O shape da imagem é (224, 224, 3)
O espectrograma salvo é no shape (224, 224, 3)
O shape da imagem é (224, 224, 3)
O espectrograma salvo é no shape (224, 224, 3)
O shape da imagem é (224, 224, 3)
O espectrograma salvo é no shape (224, 224, 3)
O shape da imagem é (224, 224, 3)
O espectrograma salvo é no shape (224, 224, 3)
O shape da imagem é (224, 224, 3)
O espectrograma salvo é no shape (224, 224, 3)
o arquivo em lote np salvo é no shape ...(224, 224, 3, 10)
O shape da imagem é (224, 224, 3)
O espectrograma salvo é no shape (224, 224, 3)
O shape da imagem é (224, 224, 3)
O espectrograma 

# Agora vamos juntar todos os vetores e salvar como uma coisa só


In [128]:
for i in range (len(subdirs)):
    file = f'gray_spectr_{folder}_{subdirs[i]}_'

    # Listar todos os arquivos que começam com 'gray_spectr_test_positive_'
    files = [f for f in os.listdir(save_dir) if f.startswith(file)]

    # Exibir os arquivos encontrados
    arrays = []
    for file in files:
        array = np.load(save_dir + file)
        arrays.append(array)

    concatenated_array = np.concatenate(arrays,axis=-1)

    #verificando se o shape esta correto
    print(concatenated_array.shape)

    # Opcional: Salvar o array concatenado em um novo arquivo .npy

    np.save(os.path.join('E:/nps/nps_gray_spec', f'{folder}_{subdirs[i]}.npy'), concatenated_array)

    print(f'{folder}_{subdirs[i]}.npy SAVED!!!')

(224, 224, 3, 281)
test_positive.npy SAVED!!!
(224, 224, 3, 281)
test_negative.npy SAVED!!!


In [129]:

# Listar todos os arquivos que começam com 'test'
files_to_delete = [f for f in os.listdir(save_dir) if f.startswith(f'gray_spectr_{folder}_')]

# Deletar cada arquivo
for file_name in files_to_delete:
    file_path = os.path.join(save_dir, file_name)
    
    if os.path.isfile(file_path):
        os.remove(file_path)
        print(f"Arquivo {file_path} deletado.")
    else:
        print(f"Arquivo {file_path} não encontrado.")


Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_test_negative_batch_1.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_test_negative_batch_10.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_test_negative_batch_11.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_test_negative_batch_12.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_test_negative_batch_13.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_test_negative_batch_14.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_ap